# Generating puzzles with multiple solutions
Puzzles with multiple solutions can create very interesting effects, such as the puzzle shown in the video "[How can a jigsaw have two distinct solutions?](https://www.youtube.com/watch?v=b5nElEbbnfU)" on the Stand-Up Maths Youtube channel where the two ways of solving the jigsaw puzzle either create an image of a coffee cup or a donut. This page describes how such puzzles can be generated by first picking a random scrambling of the puzzle pieces, and then generating a puzzle that is solvable in both its unscrambled and scrambled state. There is also Python code.

# Describing puzzles using vectors
In order for a jigsaw puzzle to be solvable in more than one way, there must be several connections between pieces that have the same shape. Otherwise, if every connection was unique, each side of a piece would only be able to connect with a single side of a single other piece, and there would only be a single solution to the puzzle. This means that each side of each puzzle piece can be grouped with other puzzle piece sides with a similar shape, and these groups can be given numbers.

It is useful to give the shapes numbers so that shapes that are compatible with each other have inverse numbers. For example, shape 1 is compatible with shape -1, shape 2 is compatible with shape -2, and so forth. This makes it easy to check whether two shapes are compatible: they are compatible if they add up to 0, and incompatible if they do not. I prefer to think of the positive numbers as all the shapes that stick out and the negative numbers as the indented shapes, but this is fully arbitrary. Finally, the outer sides of the pieces at the edge of the jigsaw puzzle are just shaped like straight lines, and do not connect to anything. These non-connecting shapes are labelled 0.

A puzzle can be described mathematically as a vector containing the shapes of each side of all the pieces of the puzzle. The order of the sides is also arbitrary, but here we order the sides in order left, top, right, bottom, starting with the top left piece and going row by row. The ordering of sides for a 2x2 puzzle is illustrated in the following figure.

<img src="sidenumbers.png" style="width:500px; margin-left:auto; margin-right:auto;"/>

Thus, if we assume that all the tabs in the image are in shape group 1 and all the indentations are in shape group -1, the vector describing this puzzle is:
$$
    \vec{p} = \begin{pmatrix}
        0 \\
        0 \\
        -1 \\
        1 \\
        1 \\
        0 \\
        0 \\
        1 \\
        0 \\
        -1 \\
        -1 \\
        0 \\
        1 \\
        -1 \\
        0 \\
        0
    \end{pmatrix}
$$

# Scramble matrices
In order to solve a puzzle, the pieces have to be moved around and rotated. I will refer to a series of movements and rotations as a scramble. These scrambles can be represented as matrices, which can be applied to the puzzle vector. A scramble matrix can be constructed as follows:
* Start with an $n\times n$ matrix with all entries set to 0, where n is the number of puzzle piece sides (and thus the length of vector $\vec{p}$.
* For each side in the puzzle, find its original position $a$ and its scrambled position $b$. Set the entry at row $b$ and column $a$ to 1.

There are a few constraints to this scramble matrix. Each row and each column in the matrix can only have a single entry that is equal to one, since all sides in the original puzzle have to be present in the scrambled puzzle, and sides can not be duplicated or destroyed. It is also not possible to move sides independently, since they are connected to puzzle pieces. This means that each $4\times4$ section of the scramble matrix can only take one of the following forms:

$$
    \mathbf{R}_0 = \begin{pmatrix}
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1
    \end{pmatrix},
    \mathbf{R}_1 = \begin{pmatrix}
        0 & 1 & 0 & 0 \\
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1 \\
        1 & 0 & 0 & 0
    \end{pmatrix},
    \mathbf{R}_2 = \begin{pmatrix}
        0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 1 \\
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0
    \end{pmatrix},
    \mathbf{R}_3 = \begin{pmatrix}
        0 & 0 & 0 & 1 \\
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 1 & 0
    \end{pmatrix},
    \mathbf{\phi} = \begin{pmatrix}
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0
    \end{pmatrix}
$$
If the submatrix starting at row $4i$ and column $4j$ in the scramble matrix looks like $\mathbf{R}_0$, then the scramble involves moving piece number $j$ to the position of piece number $i$ without rotating it. If the submatrix looks like $\mathbf{R}_1$ then piece $j$ was moved to the position of piece $i$ and rotated once counter-clockwise. Likewise, $\mathbf{R}_2$ means the piece has been moved and rotated twice, while $\mathbf{R}_3$ means the piece has been moved and rotated counter-clockwise three times (equivalent to rotating clockwise once). Submatrix $\mathbf{\phi}$ means that piece $j$ was not moved to the position of piece $i$.

As an example, the following scramble matrix involves rotating the first piece of a $2\times2$ puzzle once counter-clockwise, and then swapping the positions of the first and fourth pieces.
$$
    \mathbf{T} = \begin{pmatrix}
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
        0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
    \end{pmatrix}
$$

# Validating if a puzzle is solved
A puzzle can be validated by checking each side of the puzzle: if the side is on an outside edge of the puzzle, it must have a connection shape of 0, while if the side faces another side in the puzzle, the sum of the shapes of the two sides must be 0. This set of checks can be encoded as a verification matrix, such that a puzzle is solved if and only if its puzzle vector $\vec{p}$ satisfies the equation
$$
    \mathbf{V}\vec{p} = \vec{0}
$$
Each row in the verification matrix corresponds to checking a single side in the puzzle. If that side faces another side in the puzzle, the the column for each of those sides is set to one. If the side is on the outside edge of the puzzle, then only the column corresponding to that side is set to one. Each row that does not correspond to checking a side on the outside edge is repeated twice, since every pair of sides that face each other will have the same check. It is useful to order the rows so that row number $i$ corresponds to checking if side number $i$ is valid, since this will make each entry on the main diagonal of the matrix have the value one, and will also make the matrix symmetric about the main diagonal.

It is notable that since $\mathbf{V}\vec{p} = \vec{0}$ has many non-trivial solutions for $\vec{p}$ (any valid puzzle is a solution), the verification matrix $\mathbf{V}$ must be non-invertible.

As an example, the verification matrix for a 2x2 puzzle looks like this:
$$
    \mathbf{V} = \begin{pmatrix}
        1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 0 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
    \end{pmatrix}
$$

In [ ]:
%load verification_matrix.py

 # Finding puzzles with multiple solutions
 Any arbitrary scramble (i.e. movement and/or rotation of the pieces) of a puzzle can be described using a scramble matrix, $\mathbf{T}$. If the matrix equation $\mathbf{V}\mathbf{T}\vec{p} = \vec{0}$ holds, then this scramble is a solution to the puzzle. For a given scramble $\mathbf{T}$ and a given verification matrix $\mathbf{V}$, it is possible to construct a puzzle vector $\vec{p}$ such that the following system of matrix equations hold true:
 $$
    \mathbf{V}\mathbf{T}\vec{p} = \vec{0}
 $$
 $$
    \mathbf{V}\vec{p} = \vec{0}
 $$

A puzzle $\vec{p}$ that satisfies these constraints must have at least two solutions, since both its unscrambled state and its state that is scrambled according to $\mathbf{T}$ are solutions.

In [ ]:
%load puzzle_generator.py

# Random transformations and scramble similarity
Since we now know how to generate a puzzle that has at least two solutions given an arbitrary scramble, we can generate a random scramble and build a puzzle with multiple solutions using that. However, it is important to keep in mind that the transformation can't be completely arbitrary: only corner pieces can occupy the corners of the jigsaw puzzle, and only edge pieces can occupy the edges, and their orientation is determined by which edge they're on. All middle pieces can be arbitrarily rearranged and rotated.

In the video, Matt Parker also wants the two solutions to a puzzle to be as different as possible. One way of quantifying how similar the puzzle is after being scrambled is to count how many sides are adjacent to each other in the scrambled puzzle that were also adjacent in the original puzzle. This is easily countable: if every connecting shape in the original puzzle is unique, then every pair of sides that is still able to connect after scrambling the puzzle are sides that were connected in the original puzzle.

In [ ]:
%load scrambles.py

# Counting the number of solutions
In order to achieve the effect in the video where the puzzle motif changes from a coffee cup to a donut, the puzzle should only have two solutions, otherwise it would be possible to put it back together in different ways that create nonsensical motifs. Since the above method guarantees that a puzzle has *at least* two solutions, we need to count the number of possible solutions to ensure that the puzzle has *only* two solutions.

Counting the number of possible solutions amounts to finding the number of matrices $\mathbf{T}$ that satisfies the equation
$$
    \mathbf{V}\mathbf{T}\vec{p} = \vec{0}
$$
where $\mathbf{V}$ and $\vec{p}$ are known. Since $\mathbf{V}$ is non-invertible, I don't think there exists a better way of finding $\mathbf{T}$ than trial and error.

Since counting the number of solutions takes a long time, it is useful to weed out puzzles that obviously have more than two solutions. Any where two pieces have the exact same side shapes must have at least two solutions, since we can solve the puzzle by simply swapping those two pieces. Since we already know that our chosen puzzle also has a solution that is more complicated than just swapping two pieces, our puzzle must have more than two solutions.

It also seems likely that the more often a particular connection shape is repeated, the more likely a puzzle is to have many solutions. Additionally, puzzles with fewer repeated connections should be faster to check, and Matt Parker also states in the video that the most satisfying puzzle would be one with only two solutions, no shared sides, and no more than two of any connection type. This seems almost impossible to find, but setting an upper bound on the number of repeated connection shapes will also weed out a lot of puzzles.

In [ ]:
%load solution_finder.py

The following cell contains code to draw the puzzle

In [ ]:
%load puzzle_drawer.py

# Finding puzzles
Here, we find the verification matrix for a given width and height. Then we generate random scrambles until one is found where no two sides that were adjacent in the original puzzle are adjacent after the scramble. We then find a puzzle that is solved in both its unscrambled state and the state that results from the scramble we found. We then check that this puzzle doesn't have any connections that are repeated more than the maximum. If it does not, we check that it does not have any pieces that are duplicates of each other (i.e. have the same connections in the same order). If the puzzle meets all these conditions, we try to find all solutions for it. If the puzzle has only two solutions, it is returned, and a figure is drawn.

This is doable for $5\times5$ puzzles, seems to work for $5\times6$ puzzles, and seems to get completely bogged down for $6\times6$ puzzles. I think the solution finder needs to be made much more efficient in order to handle those.

I also suspect that the best way to generate puzzles more quickly is to find some smarter way to generate the transformations, rather than generating them randomly and checking if they meet our criteria.

Rerun all the code in this notebook to see an illustration of a puzzle with only two solutions. With the default settings, this should happen almost immediately. Lowering the value of maximum_repeated_shapes should increase the runtime, but give more satisfying puzzles with fewer connection repetitions. Increasing the value of width and height should increase the runtime drastically. I've never seen it complete for a $6\times6$ puzzle so far.

In [ ]:
%load -r 8: main.py